In [1]:
from learn import miner


/Users/lasse/Library/Mobile Documents/com~apple~CloudDocs/Studium Mannheim/2_Semester/Advanced Process Mining/Project/No-time-to-dice_group_project_IE692/OrdinoR-dev/ordinor/execution_context/rule_based/decision_tree.py:44: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self._log = el[included_cols]
/Users/lasse/Library/Mobile Documents/com~apple~CloudDocs/Studium Mannheim/2_Semester/Advanced Process Mining/Project/No-time-to-dice_group_project_IE692/OrdinoR-dev/ordinor/execution_context/rule_based/decision_tree.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._log[const.RESOURCE] = self._log[const.RESOURCE].astype('category')
/Users/lasse/Library/Mobile Documents/com~ap

Decision tree initialized with an empty root node
	Dis. = 0.000000, Imp. = 1.000000, Harmonic Mean. = 0.000000, *** Tree has 1 leaf node(s). ***
Start to fit decision tree with max. height = 3
Tree grows by splitting all current leaf nodes on `tt:weekday`, step=1 (running for 00:00:12s)
	Dis. = 0.001538, Imp. = 0.999829, Harmonic Mean. = 0.003070, *** Tree has 2 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`, step=2 (running for 00:00:24s)
	Dis. = 0.003602, Imp. = 0.998222, Harmonic Mean. = 0.007178, *** Tree has 3 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`, step=3 (running for 00:00:33s)
	Dis. = 0.003602, Imp. = 0.998144, Harmonic Mean. = 0.007179, *** Tree has 4 leaf node(s). ***
Sub-tree at step 3 is selected as the final solution: dispersal = 0.003602, impurity = 0.998144
********************************************************************************
============================== TREE SUMMARY ====================

In [2]:
from tree import TreeNode, TreePrinter
from ordinor.execution_context.rule_based.TreeNode import Node


In [3]:
all_nodes_matrix = []
all_events = []
for node in miner.l_history_persistent[0]['solution'].values():
    all_events.extend(node.event_ids)

root_node = Node(
    label=0,
    event_ids=all_events,
    ct_label=None,
    at_label=None,
    tt_label=None)
root_TreeNode_in_list = []
root_TreeNode_in_list = [TreeNode(root_node.label, root_node.parent_label, root_node, 0, None, [])]
all_nodes_matrix.append(root_TreeNode_in_list)
for i, step in enumerate(miner.l_history_persistent): # i to be sure we we are when navigating the tree
    all_nodes_this_step = []
    for node in step['solution'].values():
        current_node = TreeNode(node.label,node.parent_label, node, i+1, None, [])
        all_nodes_this_step.append(current_node)
    all_nodes_matrix.append(all_nodes_this_step)

for i, all_nodes_this_step in enumerate(reversed(all_nodes_matrix)):
    for node in all_nodes_this_step:
        if node.parent_label is not None:
            candidate_parent = next((parent_node for parent_node in all_nodes_matrix[len(
                all_nodes_matrix)-i-2] if parent_node.label == node.parent_label), None)
            if candidate_parent is not None:
                node.parent = candidate_parent
            else:
                node.parent = next((parent_node for parent_node in all_nodes_matrix[len(all_nodes_matrix)-i-2] if parent_node.label == node.label))
            if node.parent is not None:
                node.parent.children.append(node)

In [4]:

def printAllNodesWithStep(node, step):
    if node.step > step:
        return
    elif node.step == step:
        print(node)
    if node.children:
        for child in node.children:
            printAllNodesWithStep(child, step)
    else:
        return


In [5]:
for i in range(8):
    printAllNodesWithStep(all_nodes_matrix[0][0], i)
    print("-"*150)


TreeNode [0]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [339]
TreeNode [340]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [800]
TreeNode [801]
TreeNode [802]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [800]
TreeNode [1197]
TreeNode [1198]
TreeNode [802]
------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------

In [6]:
#importlib.reload(Tree)
#from tree import TreeNode, TreePrinter

In [7]:
treePrinter = TreePrinter(
    all_nodes_matrix[0][0], 
    3, 
    miner.l_history_persistent
    )
treePrinter.print_tree_svg()


In [ ]:
import cairosvg

cairosvg.svg2png(url="tree.svg", write_to="tree.png", scale=4.0)
